In [1]:
import cv2
import numpy as np
import pandas as pd
from tqdm import tqdm

In [2]:
from ultralytics import YOLO

In [3]:
model = YOLO("yolo11x-pose.pt")
model.export(format="engine")

WARNING TensorRT requires GPU export, automatically assigning device=0
Ultralytics 8.3.243  Python-3.12.9 torch-2.9.1+cu128 CUDA:0 (NVIDIA GeForce RTX 3050 Laptop GPU, 4096MiB)
YOLO11x-pose summary (fused): 199 layers, 58,751,308 parameters, 0 gradients, 202.8 GFLOPs

PyTorch: starting from 'yolo11x-pose.pt' with input shape (1, 3, 640, 640) BCHW and output shape(s) (1, 56, 8400) (113.0 MB)

ONNX: starting export with onnx 1.20.0 opset 20...
ONNX: slimming with onnxslim 0.1.80...
ONNX: export success  6.3s, saved as 'yolo11x-pose.onnx' (224.6 MB)

TensorRT: starting export with TensorRT 10.14.1.48.post1...
TensorRT: input "images" with shape(1, 3, 640, 640) DataType.FLOAT
TensorRT: output "output0" with shape(1, 56, 8400) DataType.FLOAT
TensorRT: building FP32 engine as yolo11x-pose.engine
TensorRT: export success  381.0s, saved as 'yolo11x-pose.engine' (261.0 MB)

Export complete (383.6s)
Results saved to E:\ROBOTRONIX\Learning Period\10_Pose_estimation
Predict:         yolo predict t

'yolo11x-pose.engine'

In [5]:
ENGINE_PATH = "../data/models/yolo11x-pose.engine"
VIDEO_INPUT = "../data/3196221-uhd_3840_2160_25fps.mp4"
VIDEO_OUTPUT = "../data/yolo11x_pose_tracked.mp4"
CSV_OUTPUT = "../data/yolo11x_pose_tracked.csv"

In [6]:
POSE_CONNECTIONS = [
    (5, 7), (7, 9),      # left arm
    (6, 8), (8, 10),     # right arm
    (5, 6),              # shoulders
    (5, 11), (6, 12),    # torso
    (11, 13), (13, 15),  # left leg
    (12, 14), (14, 16)   # right leg
]

In [7]:
model = YOLO(ENGINE_PATH)

In [8]:
def draw_pose(frame, kpts, track_id):
    h, w, _ = frame.shape

    for x, y, conf in kpts:
        if conf > 0.3:
            cv2.circle(frame, (int(x), int(y)), 4, (0, 0, 255), -1)

    for a, b in POSE_CONNECTIONS:
        if kpts[a][2] > 0.3 and kpts[b][2] > 0.3:
            x1, y1 = int(kpts[a][0]), int(kpts[a][1])
            x2, y2 = int(kpts[b][0]), int(kpts[b][1])
            cv2.line(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)

    if track_id is not None:
        cx, cy = int(kpts[0][0]), int(kpts[0][1])
        cv2.putText(
            frame, f"ID {track_id}",
            (cx, cy - 10),
            cv2.FONT_HERSHEY_SIMPLEX,
            0.6, (255, 255, 0), 2
        )


In [9]:
cap = cv2.VideoCapture(VIDEO_INPUT)

fps = cap.get(cv2.CAP_PROP_FPS)
w = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
h = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

fourcc = cv2.VideoWriter_fourcc(*"mp4v")
out = cv2.VideoWriter(VIDEO_OUTPUT, fourcc, fps, (w, h))


In [10]:
records = []

with tqdm(total=int(cap.get(cv2.CAP_PROP_FRAME_COUNT)), desc="Tracking") as pbar:
    while True:
        ret, frame = cap.read()
        if not ret:
            break

        results = model.track(
            frame,
            persist=True,
            conf=0.4,
            iou=0.5,
            tracker="bytetrack.yaml",
            verbose=False
        )

        if results and results[0].keypoints is not None:
            boxes = results[0].boxes
            kpts = results[0].keypoints.xy.cpu().numpy()
            confs = results[0].keypoints.conf.cpu().numpy()

            track_ids = (
                boxes.id.cpu().numpy().astype(int)
                if boxes.id is not None else [None] * len(kpts)
            )

            for i, (kp, kp_conf) in enumerate(zip(kpts, confs)):
                person_id = track_ids[i]
                kpts_full = np.hstack([kp, kp_conf[:, None]])

                draw_pose(frame, kpts_full, person_id)

                row = {
                    "frame": int(cap.get(cv2.CAP_PROP_POS_FRAMES)),
                    "track_id": person_id
                }

                for j, (x, y, c) in enumerate(kpts_full):
                    row[f"kp{j}_x"] = x
                    row[f"kp{j}_y"] = y
                    row[f"kp{j}_conf"] = c

                records.append(row)
        print(type(results[0].keypoints), results[0].keypoints is None)
        out.write(frame)
        pbar.update(1)

cap.release()
out.release()


Tracking:   0%|          | 0/385 [00:00<?, ?it/s]

Loading ../data/models/yolo11x-pose.engine for TensorRT inference...


Tracking:   1%|          | 2/385 [00:01<04:36,  1.39it/s]

<class 'ultralytics.engine.results.Keypoints'> False
<class 'ultralytics.engine.results.Keypoints'> False


Tracking:   1%|          | 4/385 [00:01<02:01,  3.13it/s]

...

Tracking:  99%|█████████▉| 383/385 [00:52<00:00,  7.38it/s]

<class 'ultralytics.engine.results.Keypoints'> False
<class 'ultralytics.engine.results.Keypoints'> False


Tracking: 100%|██████████| 385/385 [00:52<00:00,  7.38it/s]

<class 'ultralytics.engine.results.Keypoints'> False
<class 'ultralytics.engine.results.Keypoints'> False


Tracking: 100%|██████████| 385/385 [00:52<00:00,  7.35it/s]


In [11]:
df = pd.DataFrame(records)
df.to_csv(CSV_OUTPUT, index=False)

print("Done.")
print("Video:", VIDEO_OUTPUT)
print("CSV:", CSV_OUTPUT)


Done.
Video: ../data/yolo11x_pose_tracked.mp4
CSV: ../data/yolo11x_pose_tracked.csv
